# **- Capacidade de Pagamento**

Na EDA focada em capacidade de pagamento, foram analisadas variáveis relacionadas à renda, comprometimento financeiro e histórico de obrigações do cliente, buscando entender sua sustentabilidade financeira. Essa etapa permitiu identificar padrões associados ao risco de inadimplência e embasar decisões de feature engineering e modelagem.

In [ ]:
import pandas as pd

df = pd.read_csv("/content/df_final_selected_features.csv")
df.head(10)

,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,AMT_ANNUITY,AMT_CREDIT,INST_AMT_PAYMENT_MEAN,BUREAU_DAYS_CREDIT_MAX,DAYS_EMPLOYED,...,PREV_AMT_APPLICATION_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,PREV_DAYS_DECISION_MIN,POS_CNT_INSTALMENT_MEAN,POS_MONTHS_BALANCE_MIN,PREV_AMT_GOODS_PRICE_MEAN,POS_SK_DPD_MEAN,PREV_STATUS_APPROVED,BUREAU_DAYS_ENDDATE_FACT_MIN,TARGET
0,Business Entity Type 3,0.083037,0.262949,0.139376,-9461.0,24700.5,406597.5,11559.247105,-103.0,-637.0,...,179055.000000,-1072.0,-606.0,24.000000,-19.0,179055.000000,0.000000,1.0,-1185.0,1.0
1,School,0.311267,0.622246,0.535276,-16765.0,35698.5,1293502.5,64754.586000,-606.0,-1188.0,...,435436.500000,-2434.0,-2341.0,10.107143,-77.0,435436.500000,0.000000,3.0,-2131.0,0.0
2,Government,0.505998,0.555912,0.729567,-19046.0,6750.0,135000.0,7096.155000,-408.0,-225.0,...,24282.000000,-595.0,-815.0,3.750000,-27.0,24282.000000,0.000000,1.0,-683.0,0.0
3,Business Entity Type 3,0.505998,0.650442,0.535276,-19005.0,29686.5,312682.5,62947.088438,-300.0,-3039.0,...,272203.260000,-1253.0,-617.0,11.428571,-20.0,408304.890000,0.000000,5.0,-1241.0,0.0
4,Religion,0.505998,0.322738,0.535276,-19932.0,21865.5,513000.0,12214.060227,-1149.0,-3038.0,...,150530.250000,-783.0,-2357.0,15.333333,-77.0,150530.250000,0.000000,6.0,-783.0,0.0
5,Other,0.505998,0.354225,0.621226,-16941.0,27517.5,490495.5,27360.502714,-78.0,-1588.0,...,155701.800000,-853.0,-2536.0,11.518072,-84.0,194627.250000,339.060241,4.0,-1028.0,0.0
6,Business Entity Type 3,0.774761,0.724000,0.492060,-13778.0,41301.0,1560726.0,9568.531765,-239.0,-3130.0,...,76741.714286,-2152.0,-1562.0,7.875000,-96.0,76741.714286,0.000000,7.0,-2152.0,0.0
7,Other,0.505998,0.714279,0.540654,-18850.0,42075.0,1530000.0,27449.208000,-1138.0,-449.0,...,247212.000000,-928.0,-1070.0,10.000000,-35.0,247212.000000,0.000000,1.0,-1138.0,0.0
8,XNA,0.587334,0.205747,0.751724,-20099.0,33826.5,1019610.0,11328.893654,-1309.0,365243.0,...,202732.875000,-2173.0,-2508.0,19.186667,-83.0,270310.500000,205.666667,3.0,-2197.0,0.0
9,Electricity,0.505998,0.746644,0.535276,-14469.0,20250.0,405000.0,10451.285625,-300.0,-2019.0,...,60930.000000,-1253.0,-1673.0,19.021739,-55.0,81240.000000,0.000000,3.0,-1241.0,0.0


**- Análise de Renda dos clientes**

In [ ]:
# Estatísticas descritivas
df["AMT_INCOME_TOTAL"].describe()

,AMT_INCOME_TOTAL
count,3.075110e+05
mean,1.687979e+05
std,2.371231e+05
min,2.565000e+04
25%,1.125000e+05
50%,1.471500e+05
75%,2.025000e+05
max,1.170000e+08


In [ ]:
# Renda média por target
df.groupby("TARGET")["AMT_INCOME_TOTAL"].agg(["mean", "median"]).round(2)

,mean,median
TARGET,,
0.0,169077.72,148500.0
1.0,165611.76,135000.0


In [ ]:
# Criação de bins (quartis)
df["income_bin"] = pd.qcut(df["AMT_INCOME_TOTAL"], q=4, duplicates="drop")

# Taxa de inadimplência por faixa de renda
income_default = (
    df.groupby("income_bin")["TARGET"]
      .mean()
      .reset_index(name="default_rate")
).round(2)

income_default

/tmp/ipython-input-401550651.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("income_bin")["TARGET"]


,income_bin,default_rate
0,"(25649.999, 112500.0]",0.08
1,"(112500.0, 147150.0]",0.09
2,"(147150.0, 202500.0]",0.08
3,"(202500.0, 117000000.0]",0.07


A renda apresenta distribuição altamente assimétrica, com outliers que mascaram diferenças de risco quando analisada apenas pela média. A inadimplência é maior nas faixas intermediárias e não segue relação monotônica estrita com a renda. Por isso, transformações logarítmicas e discretização em faixas são essenciais para capturar padrões de risco de forma mais robusta na engenharia de features.

In [ ]:
import pandas as pd
import plotly.express as px

# Criar faixas de renda (quartis)
df["income_bin"] = pd.qcut(
    df["AMT_INCOME_TOTAL"],
    q=4,
    labels=["Muito Baixa", "Baixa", "Média", "Alta"]
)

# Calcular taxa de inadimplência
income_default = (
    df.groupby("income_bin")["TARGET"]
      .mean()
      .reset_index()
)

# Gráfico de inadimplência por faixa de renda
fig = px.bar(
    income_default,
    x="income_bin",
    y="TARGET",
    text=income_default["TARGET"].round(3),
    title="Taxa de Inadimplência por Faixa de Renda",
    labels={
        "income_bin": "Faixa de Renda",
        "TARGET": "Taxa de Inadimplência"
    }
)

# Ajustes de eixos e formatação percentual
fig.update_layout(
    yaxis_tickformat=".0%",
    xaxis_title="Faixa de Renda",
    yaxis_title="Taxa de Inadimplência"
)

fig.show()


/tmp/ipython-input-1271525869.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("income_bin")["TARGET"]


Clientes na faixa de renda mais alta apresentam menor taxa de inadimplência, enquanto faixas mais baixas concentram maior risco. A renda, quando mostrada em faixas, reduz o impacto de outliers e captura melhor o efeito não linear no risco de crédito.

In [ ]:
# Amostra do dataset para visualização
sample_df = df.sample(5000, random_state=42)

# Gráfico de dispersão: renda vs valor do crédito
fig = px.scatter(
    sample_df,
    x="AMT_INCOME_TOTAL",
    y="AMT_CREDIT",
    color="TARGET",
    opacity=0.4,
    title="Renda vs Valor do Crédito (Amostra)",
    labels={
        "AMT_INCOME_TOTAL": "Renda",
        "AMT_CREDIT": "Valor do Crédito"
    }
)

fig.show()

Nesse gráfico de dispersão podemos observar forte concentração de clientes em faixas de baixa renda e valores moderados de crédito. A relação entre renda e valor do crédito é positiva, porém não linear, indicando heterogeneidade no perfil dos clientes. A inadimplência ocorre em diferentes níveis de renda e crédito, reforçando o caráter multifatorial do risco de crédito.

**- Análise de endividamento**

In [ ]:
# Criando uma variável para mostrar quanto de crédito representa a renda anual do cliente
df["DEBT_INCOME_RATIO"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]

In [ ]:
# Compara média e mediana do índice de endividamento entre clientes adimplentes e inadimplentes
df.groupby("TARGET")["DEBT_INCOME_RATIO"].agg(
    mean_ratio="mean",
    median_ratio="median"
)

,mean_ratio,median_ratio
TARGET,,
0.0,3.963729,3.266653
1.0,3.887438,3.253143


A análise total mostra pouca diferença no endividamento médio entre adimplentes e inadimplentes, indicando ausência de relação linear direta. Isso sugere que o risco se manifesta apenas em faixas específicas ou interações com outras variáveis. Portanto, o endividamento deve ser explorado via discretização, thresholds ou interações, e não como variável contínua simples.

In [ ]:
# Cria faixas de endividamento com base no debt-to-income ratio
df["debt_ratio_bin"] = pd.qcut(
    df["DEBT_INCOME_RATIO"],
    q=4,
    labels=["Baixo", "Médio", "Alto", "Muito Alto"]
)

# Calcula inadimplência média por faixa de endividamento
debt_default = (
    df.groupby("debt_ratio_bin")["TARGET"]
      .mean()
      .reset_index(name="default_rate")
)

# Gráfico de inadimplência por faixa de endividamento
fig = px.bar(
    debt_default,
    x="debt_ratio_bin",
    y="default_rate",
    text="default_rate",
    labels={
        "debt_ratio_bin": "Faixa de Endividamento",
        "default_rate": "Taxa de Inadimplência"
    },
    title="Taxa de Inadimplência por Faixa de Endividamento"
)

# Formata rótulos das barras como percentual
fig.update_traces(
    texttemplate="%{y:.1%}",
    textposition="outside"
)

# Formata eixo Y como percentual
fig.update_layout(
    yaxis_tickformat=".0%"
)

fig.show()


/tmp/ipython-input-3424843042.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Analisando os gráficos podemos observar que a inadimplência é maior nas faixas intermediárias de endividamento, enquanto clientes extremamente endividados apresentam menor risco, indicando forte efeito de seleção de crédito. Isso evidencia uma relação não linear entre endividamento e risco. A variável deve ser usada via discretização ou combinada com outras features para capturar esse comportamento.

**- Prestação Mensal × Renda**

In [ ]:
# Criar razão entre prestação mensal e renda total
df["ANNUITY_INCOME_RATIO"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]

# Ver estatísticas básicas da nova variável
df["ANNUITY_INCOME_RATIO"].describe().round(2)


,ANNUITY_INCOME_RATIO
count,307511.00
mean,0.18
std,0.09
min,0.00
25%,0.11
50%,0.16
75%,0.23
max,1.88


In [ ]:
# Comparar média e mediana por status de inadimplência
df.groupby("TARGET")["ANNUITY_INCOME_RATIO"].agg(
    mean_ratio="mean",
    median_ratio="median"
).round(2)

,mean_ratio,median_ratio
TARGET,,
0.0,0.18,0.16
1.0,0.19,0.17


In [ ]:
# Criar faixas de comprometimento usando quartis
df["ANNUITY_RATIO_BIN"] = pd.qcut(
    df["ANNUITY_INCOME_RATIO"],
    q=4,
    labels=["Baixo", "Médio", "Alto", "Muito Alto"]
)


In [ ]:
import plotly.express as px

# Calcular taxa de inadimplência por faixa
annuity_default = (
    df.groupby("ANNUITY_RATIO_BIN")["TARGET"]
      .mean()
      .reset_index(name="DEFAULT_RATE")
)

# Criar gráfico de barras
fig = px.bar(
    annuity_default,
    x="ANNUITY_RATIO_BIN",
    y="DEFAULT_RATE",
    text="DEFAULT_RATE",
    title="Taxa de Inadimplência por Faixa de Comprometimento Mensal",
    labels={
        "ANNUITY_RATIO_BIN": "Faixa de Comprometimento",
        "DEFAULT_RATE": "Taxa de Inadimplência"
    }
)

# Ajustar formato do texto e eixo Y
fig.update_traces(
    texttemplate="%{y:.1%}",
    textposition="outside"
)

fig.update_layout(
    yaxis_tickformat=".0%"
)

# Exibir gráfico
fig.show()


/tmp/ipython-input-2697452494.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



A inadimplência aumenta conforme cresce o comprometimento mensal da renda, indicando relação direta entre estresse financeiro e risco de crédito. Essa variável captura melhor a capacidade de pagamento real do que renda ou endividamento total. Por isso, é uma das features mais relevantes para modelos de risco.